<a href="https://colab.research.google.com/github/ianellisjones/usn/blob/main/Fleet_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
US NAVY FLEET TRACKER (IEJ)

A scraping utility to track the deployment history of the entire US Navy Surface Fleet.
Covers: CVN, LHA, LHD, LPD, LSD, LCC, CG, LCS, AS, ESB, and DDG.
"""

import csv
import re
from pathlib import Path
from typing import List, Tuple, Dict

import requests
from bs4 import BeautifulSoup

# --- CONFIGURATION ---

FLEET_URLS: List[str] = [
    # Aircraft Carriers (CVN)
    "http://uscarriers.net/cvn68history.htm", "http://uscarriers.net/cvn69history.htm",
    "http://uscarriers.net/cvn70history.htm", "http://uscarriers.net/cvn71history.htm",
    "http://uscarriers.net/cvn72history.htm", "http://uscarriers.net/cvn73history.htm",
    "http://uscarriers.net/cvn74history.htm", "http://uscarriers.net/cvn75history.htm",
    "http://uscarriers.net/cvn76history.htm", "http://uscarriers.net/cvn77history.htm",
    "http://uscarriers.net/cvn78history.htm",

    # Amphibious Assault (LHA/LHD)
    "http://uscarriers.net/lhd1history.htm", "http://uscarriers.net/lhd2history.htm",
    "http://uscarriers.net/lhd3history.htm", "http://uscarriers.net/lhd4history.htm",
    "http://uscarriers.net/lhd5history.htm", "http://uscarriers.net/lhd7history.htm",
    "http://uscarriers.net/lhd8history.htm", "http://uscarriers.net/lha6history.htm",
    "http://uscarriers.net/lha7history.htm",

    # Transport Docks (LPD)
    "http://uscarriers.net/lpd17history.htm", "http://uscarriers.net/lpd18history.htm",
    "http://uscarriers.net/lpd19history.htm", "http://uscarriers.net/lpd20history.htm",
    "http://uscarriers.net/lpd21history.htm", "http://uscarriers.net/lpd22history.htm",
    "http://uscarriers.net/lpd23history.htm", "http://uscarriers.net/lpd24history.htm",
    "http://uscarriers.net/lpd25history.htm", "http://uscarriers.net/lpd26history.htm",
    "http://uscarriers.net/lpd27history.htm", "http://uscarriers.net/lpd28history.htm",
    "http://uscarriers.net/lpd29history.htm",

    # Dock Landing Ships (LSD)
    "http://uscarriers.net/lsd44history.htm", "http://uscarriers.net/lsd46history.htm",
    "http://uscarriers.net/lsd50history.htm", "http://uscarriers.net/lsd51history.htm",
    "http://uscarriers.net/lsd42history.htm", "http://uscarriers.net/lsd45history.htm",
    "http://uscarriers.net/lsd47history.htm", "http://uscarriers.net/lsd48history.htm",
    "http://uscarriers.net/lsd49history.htm", "http://uscarriers.net/lsd52history.htm",

    # Command Ships (LCC)
    "http://uscarriers.net/lcc19history.htm", "http://uscarriers.net/lcc20history.htm",

    # Submarine Tenders (AS)
    "http://uscarriers.net/as39history.htm", "http://uscarriers.net/as40history.htm",

    # Expeditionary Sea Bases (ESB)
    "http://uscarriers.net/esb3history.htm", "http://uscarriers.net/esb4history.htm",
    "http://uscarriers.net/esb5history.htm", "http://uscarriers.net/esb6history.htm",

    # Guided Missile Cruisers (CG)
    "http://uscarriers.net/cg58history.htm", "http://uscarriers.net/cg60history.htm",
    "http://uscarriers.net/cg64history.htm", "http://uscarriers.net/cg67history.htm",
    "http://uscarriers.net/cg59history.htm", "http://uscarriers.net/cg65history.htm",
    "http://uscarriers.net/cg70history.htm", "http://uscarriers.net/cg71history.htm",
    "http://uscarriers.net/cg62history.htm",

    # Littoral Combat Ships (LCS)
    "http://uscarriers.net/lcs3history.htm", "http://uscarriers.net/lcs6history.htm",
    "http://uscarriers.net/lcs8history.htm", "http://uscarriers.net/lcs10history.htm",
    "http://uscarriers.net/lcs12history.htm", "http://uscarriers.net/lcs14history.htm",
    "http://uscarriers.net/lcs16history.htm", "http://uscarriers.net/lcs18history.htm",
    "http://uscarriers.net/lcs20history.htm", "http://uscarriers.net/lcs22history.htm",
    "http://uscarriers.net/lcs24history.htm", "http://uscarriers.net/lcs26history.htm",
    "http://uscarriers.net/lcs28history.htm", "http://uscarriers.net/lcs30history.htm",
    "http://uscarriers.net/lcs32history.htm", "http://uscarriers.net/lcs34history.htm",
    "http://uscarriers.net/lcs36history.htm", "http://uscarriers.net/lcs13history.htm",
    "http://uscarriers.net/lcs15history.htm", "http://uscarriers.net/lcs17history.htm",
    "http://uscarriers.net/lcs19history.htm", "http://uscarriers.net/lcs21history.htm",
    "http://uscarriers.net/lcs23history.htm", "http://uscarriers.net/lcs25history.htm",
    "http://uscarriers.net/lcs27history.htm", "http://uscarriers.net/lcs29history.htm",

    # Guided Missile Destroyers (DDG)
    "http://uscarriers.net/ddg51history.htm", "http://uscarriers.net/ddg52history.htm",
    "http://uscarriers.net/ddg53history.htm", "http://uscarriers.net/ddg54history.htm",
    "http://uscarriers.net/ddg55history.htm", "http://uscarriers.net/ddg56history.htm",
    "http://uscarriers.net/ddg57history.htm", "http://uscarriers.net/ddg58history.htm",
    "http://uscarriers.net/ddg59history.htm", "http://uscarriers.net/ddg60history.htm",
    "http://uscarriers.net/ddg61history.htm", "http://uscarriers.net/ddg62history.htm",
    "http://uscarriers.net/ddg63history.htm", "http://uscarriers.net/ddg64history.htm",
    "http://uscarriers.net/ddg65history.htm", "http://uscarriers.net/ddg66history.htm",
    "http://uscarriers.net/ddg67history.htm", "http://uscarriers.net/ddg68history.htm",
    "http://uscarriers.net/ddg69history.htm", "http://uscarriers.net/ddg70history.htm",
    "http://uscarriers.net/ddg71history.htm", "http://uscarriers.net/ddg72history.htm",
    "http://uscarriers.net/ddg73history.htm", "http://uscarriers.net/ddg74history.htm",
    "http://uscarriers.net/ddg75history.htm", "http://uscarriers.net/ddg76history.htm",
    "http://uscarriers.net/ddg77history.htm", "http://uscarriers.net/ddg78history.htm",
    "http://uscarriers.net/ddg79history.htm", "http://uscarriers.net/ddg80history.htm",
    "http://uscarriers.net/ddg81history.htm", "http://uscarriers.net/ddg82history.htm",
    "http://uscarriers.net/ddg83history.htm", "http://uscarriers.net/ddg84history.htm",
    "http://uscarriers.net/ddg85history.htm", "http://uscarriers.net/ddg86history.htm",
    "http://uscarriers.net/ddg87history.htm", "http://uscarriers.net/ddg88history.htm",
    "http://uscarriers.net/ddg89history.htm", "http://uscarriers.net/ddg90history.htm",
    "http://uscarriers.net/ddg91history.htm", "http://uscarriers.net/ddg92history.htm",
    "http://uscarriers.net/ddg93history.htm", "http://uscarriers.net/ddg94history.htm",
    "http://uscarriers.net/ddg95history.htm", "http://uscarriers.net/ddg96history.htm",
    "http://uscarriers.net/ddg97history.htm", "http://uscarriers.net/ddg98history.htm",
    "http://uscarriers.net/ddg99history.htm", "http://uscarriers.net/ddg100history.htm",
    "http://uscarriers.net/ddg101history.htm", "http://uscarriers.net/ddg102history.htm",
    "http://uscarriers.net/ddg103history.htm", "http://uscarriers.net/ddg104history.htm",
    "http://uscarriers.net/ddg105history.htm", "http://uscarriers.net/ddg106history.htm",
    "http://uscarriers.net/ddg107history.htm", "http://uscarriers.net/ddg108history.htm",
    "http://uscarriers.net/ddg109history.htm", "http://uscarriers.net/ddg110history.htm",
    "http://uscarriers.net/ddg111history.htm", "http://uscarriers.net/ddg112history.htm",
    "http://uscarriers.net/ddg113history.htm", "http://uscarriers.net/ddg114history.htm",
    "http://uscarriers.net/ddg115history.htm", "http://uscarriers.net/ddg116history.htm",
    "http://uscarriers.net/ddg117history.htm", "http://uscarriers.net/ddg118history.htm",
    "http://uscarriers.net/ddg119history.htm", "http://uscarriers.net/ddg120history.htm",
    "http://uscarriers.net/ddg121history.htm", "http://uscarriers.net/ddg122history.htm",
    "http://uscarriers.net/ddg123history.htm", "http://uscarriers.net/ddg125history.htm",
    # Zumwalt Class
    "http://uscarriers.net/ddg1000history.htm", "http://uscarriers.net/ddg1001history.htm",
    "http://uscarriers.net/ddg1002history.htm",
]

OUTPUT_FILENAME = "fleet_status.csv"
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'


def fetch_history_text(url: str, char_limit: int = 50000) -> str:
    try:
        response = requests.get(url, headers={'User-Agent': USER_AGENT}, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        full_text = soup.get_text(separator='\n')
        lines = [line.strip() for line in full_text.split('\n') if line.strip()]
        clean_text = '\n'.join(lines)
        return clean_text[-char_limit:] if len(clean_text) > char_limit else clean_text
    except requests.RequestException as e:
        return f"ERROR: {str(e)}"


def parse_status_entry(text_block: str) -> Tuple[str, str]:
    lines = text_block.split('\n')

    current_year = "Unknown"
    years_found = re.findall(r'(202[3-6])', text_block)

    if years_found:
        priority_years = [y for y in years_found if y in ['2024', '2025']]
        current_year = priority_years[-1] if priority_years else years_found[-1]

    processed_lines = []
    running_year = current_year

    for line in lines:
        year_match = re.search(r'^202[3-6]', line)
        if year_match:
            running_year = year_match.group(0)
        processed_lines.append({'text': line, 'year': running_year})

    keywords = [
        "moored", "anchored", "underway", "arrived", "departed",
        "transited", "operations", "returned", "participated", "conducted",
        "moved to", "visited", "pulled into", "sea trials", "flight deck certification",
        "undocked", "dry-dock"
    ]

    allowed_years = ["2024", "2025", "2026"]

    for entry in reversed(processed_lines):
        text_lower = entry['text'].lower()
        year = entry['year']

        if year in allowed_years and any(k in text_lower for k in keywords):
            if text_lower.strip().startswith("from ") and " - " in text_lower:
                continue
            return year, entry['text']

    return current_year, "No status found."


def categorize_location(text: str) -> str:
    """
    Maps status text to standardized high-level location tags.
    Logic: Last Occurrence Wins (Sorts by End Index).
    """
    text = text.lower()

    location_map = {
        # Movement Overrides
        "Pacific Ocean": ["departed san diego", "departed pearl harbor", "pacific ocean", "pacific", "departed bremerton", "departed everett", "departed seattle", "north pacific ocean"],
        "Atlantic Ocean": ["departed norfolk", "departed mayport", "atlantic ocean", "atlantic", "departed little creek", "off the coast of panama city", "north atlantic ocean"],

        # Major US Ports
        "San Diego": ["san diego", "north island", "camp pendleton", "nassco", "bae systems", "point loma", "a-175", "a-173", "seal beach"],
        "Norfolk": ["norfolk", "portsmouth", "virginia beach", "newport news", "little creek"],
        "Mayport": ["mayport", "jacksonville"],
        "Bremerton": ["bremerton", "kitsap"],
        "Everett": ["everett"],
        "Pearl Harbor": ["pearl harbor"],
        "Seattle": ["seattle", "vigor shipyard"],
        "Portland, OR": ["swan island", "portland", "oregon"],
        "Groton": ["groton"],
        "Kings Bay": ["kings bay"],
        "Maine": ["maine", "bar harbor"],

        # Shipyards/Industrial
        "Pascagoula": ["pascagoula", "ingalls"],
        "Mobile, AL": ["mobile", "austal"],
        "Pensacola": ["pensacola"],
        "Marinette": ["marinette"],
        "Philadelphia": ["philadelphia"],
        "Orange, TX": ["orange, texas", "orange, tx"],
        "Charleston, SC": ["charleston"],
        "Port Canaveral": ["port canaveral"],
        "Port Angeles": ["port angeles"],

        # Foreign Ports
        "Yokosuka": ["yokosuka"],
        "Sasebo": ["sasebo", "juliet basin"],
        "Yokohama": ["yokohama"],
        "Busan": ["busan", "south korea"],
        "Guam": ["guam", "apra"],
        "Singapore": ["singapore", "changi"],
        "Bahrain": ["bahrain", "manama"],
        "Dubai": ["dubai", "jebel ali"],
        "Qatar": ["qatar", "emiri", "mesaieed"],
        "Duqm": ["duqm", "d'uqm", "oman"],
        "Manila": ["manila", "subic", "philippines"],
        "Klang": ["malaysia", "klang"],
        "Brunei": ["brunei", "muara"],
        "Okinawa": ["okinawa"],
        "Thailand": ["gulf of thailand", "pattaya"],
        "Australia": ["perth", "fremantle", "brisbane", "sydney"],
        "Rota": ["rota", "spain"],
        "Italy": ["italy", "genoa", "naples", "augusta bay", "gaeta"],
        "Malta": ["valletta", "malta"],
        "Saipan": ["saipan"],
        "Diego Garcia": ["diego garcia"],
        "Souda Bay": ["souda bay", "crete"],

        # Regions
        "South China Sea": ["south china sea", "spratly", "luzon"],
        "Western Pacific": ["western pacific", "westpac", "san bernardino", "philippine sea", "sea of japan"],
        "Persian Gulf": ["persian gulf", "arabian gulf"],
        "Arabian Sea": ["arabian sea"],
        "Red Sea": ["red sea", "bab el-mandeb"],
        "Gulf of Oman": ["gulf of oman"],
        "Gulf of Aden": ["gulf of aden"],
        "Mediterranean": ["mediterranean", "adriatic", "gibraltar", "med sea"],
        "Caribbean Sea": ["caribbean", "st. croix", "trinidad", "tobago", "puerto rico", "virgin islands", "st. thomas"],
        "North Sea": ["north sea", "norwegian sea", "oslo"],
        "Suez Canal": ["suez"],
        "Panama Canal": ["panama canal"],

        # Oceans
        "Indian Ocean": ["indian ocean"],
    }

    matches = []

    for label, keywords in location_map.items():
        for k in keywords:
            start_index = text.rfind(k)
            if start_index != -1:
                end_index = start_index + len(k)
                matches.append((end_index, len(k), label))

    if matches:
        matches.sort(key=lambda x: (x[0], x[1]), reverse=True)
        return matches[0][2]

    return "Underway / Unknown"


def extract_date(text: str) -> str:
    pattern = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}'
    matches = re.findall(pattern, text, re.IGNORECASE)
    return matches[-1] if matches else "Date Unspecified"


def main():
    print(f"{'='*90}")
    print(f"US NAVY TOTAL FLEET TRACKER (COMPLETE SURFACE FORCE)")
    print(f"{'='*90}\n")

    results: List[Dict[str, str]] = []

    for url in FLEET_URLS:
        # Updated Regex to include DDG
        hull_match = re.search(r'((?:cvn|lha|lhd|lpd|lsd|lcc|cg|lcs|as|esb|ddg)\d+)', url, re.IGNORECASE)
        hull = hull_match.group(1).upper() if hull_match else "UNK"

        raw_text = fetch_history_text(url)

        if "ERROR" in raw_text:
            year, status, loc_tag, date_str = "Error", raw_text, "Error", "Error"
        else:
            year, status = parse_status_entry(raw_text)
            loc_tag = categorize_location(status)
            date_str = extract_date(status)

            if date_str == "Date Unspecified":
                date_str = year

        results.append({
            "Hull": hull,
            "Location": loc_tag,
            "Date": date_str,
            "Status Sentence": status,
            "Source URL": url
        })

        print(f"[{hull}] [{loc_tag}] [{date_str}] {status}")

    try:
        output_path = Path(OUTPUT_FILENAME)
        with output_path.open(mode='w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ["Hull", "Location", "Date", "Status Sentence", "Source URL"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(results)

        print(f"\n{'='*90}")
        print(f"SUCCESS: Report saved to '{output_path.absolute()}'")
        print(f"{'='*90}")

    except PermissionError:
        print(f"\nERROR: Could not write to {OUTPUT_FILENAME}. Is the file open?")

if __name__ == "__main__":
    main()

US NAVY TOTAL FLEET TRACKER (COMPLETE SURFACE FORCE)

[CVN68] [Western Pacific] [Nov. 18] From November 8-11, the Nimitz CSG conducted operations  off the coast of Brunei; Conducted operations northeast of Spratly Islands from Nov. 12-13; Conducted operations off the southwest coast of Luzon from Nov. 14-17; Transited the San Bernardino Strait northbound on Nov. 18.
[CVN69] [Norfolk] [Jan. 8] September 26, USS Dwight D. Eisenhower moored at Pier 12N on Naval Station Norfolk after a six-day underway for TRACOM-CQ, in the Jacksonville Op. Area; Moved "dead-stick" to Super Pier 5N in Norfolk Naval Shipyard, for a Planned Incremental Availability (PIA), on Jan. 8.
[CVN70] [San Diego] [September 16] September 16, The Carl Vinson moored at Juliet Pier on Naval Air Station North Island after a three-day underway for ammo offload.
[CVN71] [San Diego] [Nov. 10] November  8, USS Theodore Roosevelt moored at Berth Lima on Naval Air Station North Island   after a 12-day underway for Tailored Ship'

In [3]:
"""
US NAVY FLEET MAPPER (v13.0 - Complete Surface Fleet)

Visualizes the output of the 'Fleet Tracker' (fleet_status.csv).
"""

import pandas as pd
import folium
import os
from folium.plugins import MarkerCluster, BeautifyIcon

# --- CONFIGURATION ---
INPUT_FILE = "fleet_status.csv"
OUTPUT_FILE = "fleet_map.html"

# --- COORDINATE DICTIONARY ---
LOCATION_COORDS = {
    # US Ports / Shipyards
    "norfolk": [36.96, -76.30],
    "san diego": [32.68, -117.18],
    "mayport": [30.39, -81.42],
    "pearl harbor": [21.35, -157.97],
    "bremerton": [47.55, -122.64],
    "kitsap": [47.70, -122.70],
    "everett": [47.98, -122.22],
    "seattle": [47.60, -122.33],
    "port angeles": [48.12, -123.43],
    "portland, or": [45.56, -122.71],
    "groton": [41.39, -72.08],
    "kings bay": [30.80, -81.53],
    "pascagoula": [30.34, -88.56],
    "mobile, al": [30.69, -88.04],
    "pensacola": [30.35, -87.27],
    "panama city": [30.15, -85.75],
    "marinette": [45.10, -87.62],
    "orange, tx": [30.09, -93.73],
    "philadelphia": [39.89, -75.17],
    "charleston, sc": [32.85, -79.97],
    "port canaveral": [28.41, -80.60],
    "maine": [43.90, -69.80], # Bath/General Maine

    # Foreign Ports
    "yokosuka": [35.29, 139.66],
    "yokohama": [35.44, 139.63],
    "sasebo": [33.16, 129.71],
    "busan": [35.10, 129.11],
    "guam": [13.44, 144.65],
    "saipan": [15.21, 145.72],
    "okinawa": [26.30, 127.80],
    "singapore": [1.30, 103.85],
    "klang": [3.00, 101.38],
    "brunei": [4.53, 114.72],
    "manila": [14.59, 120.97],
    "subic": [14.80, 120.27],
    "thailand": [12.60, 100.90],
    "bahrain": [26.22, 50.61],
    "dubai": [25.26, 55.30],
    "qatar": [25.35, 51.55],
    "duqm": [19.66, 57.70],
    "diego garcia": [-7.31, 72.41],
    "australia": [ -32.05, 115.74], # Perth
    "rota": [36.62, -6.35],
    "italy": [40.84, 14.25], # Naples/Gaeta general
    "gaeta": [41.21, 13.58],
    "souda bay": [35.48, 24.07],
    "malta": [35.90, 14.51],

    # Regions
    "south china sea": [12.00, 114.00],
    "philippine sea": [20.00, 130.00],
    "western pacific": [15.00, 135.00],
    "sea of japan": [40.00, 135.00],
    "persian gulf": [27.00, 51.00],
    "arabian sea": [15.00, 65.00],
    "red sea": [20.00, 38.00],
    "gulf of oman": [24.00, 58.00],
    "gulf of aden": [12.00, 48.00],
    "mediterranean": [35.00, 18.00],
    "caribbean sea": [15.00, -75.00],
    "north sea": [56.00, 3.00],
    "strait of gibraltar": [35.95, -5.60],
    "suez canal": [30.60, 32.33],
    "panama canal": [9.10, -79.67],
    "bab el-mandeb": [12.58, 43.33],

    # Oceans
    "atlantic ocean": [33.00, -60.00],
    "pacific ocean": [25.00, -150.00],
    "indian ocean": [-5.00, 80.00],
    "underway / unknown": [0.0, 0.0]
}

def get_location_key_and_coords(text):
    if not isinstance(text, str): return "unknown", LOCATION_COORDS["underway / unknown"]
    text = text.lower()
    if text in LOCATION_COORDS: return text, LOCATION_COORDS[text]
    for location in LOCATION_COORDS:
        if location in text: return location, LOCATION_COORDS[location]
    return "unknown", LOCATION_COORDS["underway / unknown"]

def generate_map():
    print(f"{'='*60}")
    print(f"OFFICIAL FLEET MAPPER (Total Fleet)")
    print(f"{'='*60}")

    if not os.path.exists(INPUT_FILE):
        print(f"ERROR: {INPUT_FILE} not found. Run fleet_scraper.py first.")
        return

    locations_db = {}

    def add_to_db(loc_key, coords, ship_data):
        if loc_key in ["newport news", "portsmouth", "little creek"]: loc_key = "norfolk"
        if loc_key not in locations_db:
            locations_db[loc_key] = {"coords": coords, "ships": []}
        locations_db[loc_key]["ships"].append(ship_data)

    print(f"Reading fleet data from {INPUT_FILE}...")
    df = pd.read_csv(INPUT_FILE)

    mapped_count = 0

    for _, row in df.iterrows():
        loc_tag = str(row['Location'])
        loc_key, coords = get_location_key_and_coords(loc_tag)
        hull = row['Hull'].upper()

        # Ship Class Determination
        if "CVN" in hull: s_type = "CVN"
        elif any(x in hull for x in ["LHA", "LHD", "LPD", "LSD"]): s_type = "AMPHIB"
        elif "LCC" in hull: s_type = "COMMAND"
        elif "CG" in hull: s_type = "CRUISER"
        elif "LCS" in hull: s_type = "LCS"
        elif "AS" in hull: s_type = "TENDER"
        elif "ESB" in hull: s_type = "ESB"
        elif "DDG" in hull: s_type = "DESTROYER"
        else: s_type = "OTHER"

        add_to_db(loc_key, coords, {
            "hull": hull,
            "date": row['Date'],
            "status": row['Status Sentence'],
            "type": s_type
        })
        mapped_count += 1

    m = folium.Map(location=[25, 0], zoom_start=3, tiles="CartoDB positron")

    for loc_key, data in locations_db.items():
        coords = data['coords']
        ships = data['ships']

        # Icon Priority
        if any(s['type'] == "CVN" for s in ships): color, icon = "blue", "plane"
        elif any(s['type'] == "AMPHIB" for s in ships): color, icon = "red", "anchor"
        elif any(s['type'] == "COMMAND" for s in ships): color, icon = "purple", "flag"
        elif any(s['type'] == "DESTROYER" for s in ships): color, icon = "black", "rocket"
        elif any(s['type'] == "CRUISER" for s in ships): color, icon = "darkgreen", "crosshairs"
        elif any(s['type'] == "ESB" for s in ships): color, icon = "cadetblue", "ship"
        elif any(s['type'] == "TENDER" for s in ships): color, icon = "gray", "wrench"
        else: color, icon = "orange", "ship"

        if coords == [0.0, 0.0]: color, icon, loc_label = "gray", "question", "LOCATION UNKNOWN"
        else: loc_label = loc_key.title()

        hull_list = [s['hull'] for s in ships]
        tooltip_text = f"{loc_label}: {len(ships)} Ships"

        popup_html = f"""
        <div style="font-family:sans-serif; width:350px; overflow-y:auto; max-height:300px;">
            <h4 style="margin-bottom:5px; border-bottom:2px solid {color};">{loc_label.upper()}</h4>
            <table style="width:100%; font-size:11px; border-collapse:collapse;">
        """

        # Sort: CVN > LCC > Amphib > CG > DDG > ESB > AS > LCS
        ships.sort(key=lambda x: {"CVN":1, "COMMAND":2, "AMPHIB":3, "CRUISER":4, "DESTROYER":5, "ESB":6, "TENDER":7, "LCS":8, "OTHER":9}.get(x['type'], 99))

        for s in ships:
            t_col = {"CVN": "darkblue", "AMPHIB": "darkred", "COMMAND": "purple", "CRUISER": "green", "DESTROYER": "black", "LCS": "orange", "ESB": "teal", "TENDER": "gray"}.get(s['type'], "black")
            popup_html += f"""
            <tr style="border-bottom:1px solid #eee;">
                <td style="padding:5px; vertical-align:top; width:70px;"><b style="color:{t_col};">{s['hull']}</b></td>
                <td style="padding:5px;"><span style="color:gray; font-size:9px;">{s['date']}</span><br>{s['status'][:200]}</td>
            </tr>
            """
        popup_html += "</table></div>"

        folium.Marker(location=coords, popup=folium.Popup(popup_html, max_width=370), tooltip=tooltip_text, icon=folium.Icon(color=color, icon=icon, prefix='fa')).add_to(m)

    title_html = '''
             <div style="position: fixed; top: 10px; right: 10px; width: 180px; height: 220px; z-index:9999; font-size:11px; background-color: white; opacity: 0.9; padding: 10px; border-radius: 5px; border: 1px solid grey;">
                 <b>US Navy Fleet Overview</b><br><hr style="margin:5px 0;">
                 <i class="fa fa-plane" style="color:blue; margin-right:5px;"></i> Carrier Group<br>
                 <i class="fa fa-flag" style="color:purple; margin-right:5px;"></i> Command Ship<br>
                 <i class="fa fa-anchor" style="color:red; margin-right:5px;"></i> Amphibious Group<br>
                 <i class="fa fa-rocket" style="color:black; margin-right:5px;"></i> Destroyer (DDG)<br>
                 <i class="fa fa-ship" style="color:cadetblue; margin-right:5px;"></i> Sea Base (ESB)<br>
                 <i class="fa fa-crosshairs" style="color:darkgreen; margin-right:5px;"></i> Cruiser<br>
                 <i class="fa fa-wrench" style="color:gray; margin-right:5px;"></i> Tender (AS)<br>
                 <i class="fa fa-ship" style="color:orange; margin-right:5px;"></i> LCS / Other
             </div>
             '''
    m.get_root().html.add_child(folium.Element(title_html))
    m.save(OUTPUT_FILE)
    print(f"\nSUCCESS: Map generated at '{os.path.abspath(OUTPUT_FILE)}'")
    print(f"Mapped {mapped_count} ships to {len(locations_db)} locations.")

if __name__ == "__main__":
    generate_map()

OFFICIAL FLEET MAPPER (Total Fleet)
Reading fleet data from fleet_status.csv...

SUCCESS: Map generated at '/content/fleet_map.html'
Mapped 163 ships to 29 locations.
